버스 정류장 위치 선정 공모전입니다  
개인적으로 분석 부분은 열심히 진행했다고 생각하지만 예선에서 떨어졌습니다  
그 이유에 대해 고민을 해보니 주제에 대해 명확히 고민을 하지 않고
분석적인 내용이 너무 방대하다 보니 오히려 독이 된 것 같습니다  
일을 시작하기 전에 고민을 충분히 하고 현명하게 노력해야 된다는 교훈을 얻었습니다

# Preprocess about bus data
1. 버스정류장(약 1100여개)에서의 시간대별 유동인구
2. 버스정류장(약 1100여개)에서의 연령별 유동인구

In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm

In [ ]:
import json
import folium
from folium.plugins import HeatMap

import cufflinks as cf
import plotly.offline
cf.go_offline()

## Import data

In [ ]:
bus_stop = pd.read_csv('./data/1.수원시_버스정류장.csv')
flow_time = pd.read_csv('./data/14.수원시_시간대별_유동인구(2020).csv')
flow_age = pd.read_csv('./data/15.수원시_성연령별_유동인구(2020).csv')

## 시간대, 연령별 유동인구 결합


In [ ]:
flow_time.groupby(['lon', 'lat']).size().reset_index()  # 각각의 위치에서 count 찾기 : 모두 12가 나와야되지만 곳곳에 빠진 값이 있음
# 이는 연령도 마찬가지

,lon,lat,0
0,126.927348,37.278703,4
1,126.927389,37.273295,10
2,126.927392,37.272844,4
3,126.927395,37.272393,10
4,126.927399,37.271943,10
...,...,...,...
38035,127.089116,37.297403,4
38036,127.089126,37.295600,11
38037,127.089129,37.295149,12
38038,127.089134,37.294248,12


In [ ]:
flow_time_grouped = flow_time.groupby(['lon', 'lat']).mean().reset_index()  # 한개의 좌표값당 12개의 값이 있으므로 평균값을 사용
flow_age_grouped = flow_age.groupby(['lon', 'lat']).mean().reset_index()

In [ ]:
def distance_kilometer(df, coordinate):
    """
    두개의 좌표 : (lon, lat), (lon, lat)가 주어졌을 때, 두 좌표 사이의 거리를 반환
    ref : https://ko.wikipedia.org/wiki/%EC%A7%80%EB%A6%AC%EC%A2%8C%ED%91%9C_%EA%B1%B0%EB%A6%AC
    https://lovestudycom.tistory.com/entry/%EC%9C%84%EB%8F%84-%EA%B2%BD%EB%8F%84-%EA%B3%84%EC%82%B0%EB%B2%95
    """
    lon = coordinate[0]
    lat = coordinate[1]
    return (((df['lon'] - lon) * 88.9) ** 2 + ((df['lat'] - lat) * 111) ** 2) ** (1/2)
    

In [ ]:
def preprocess(df, bus_stop, kilometer):
    """
    df : 유동인구 데이터(연령별 or 시간대별)
    bus_stop : 버스정류소 위치 데이터
    kilometer : 반경 몇 km까지 검사할 것인지
    """
    column_names = df.columns[3:].values
    column_value_arr = []
    
    for lon, lat in tqdm(bus_stop[['lon','lat']].values):
        coordinate = (lon, lat)

        column_value = df[(distance_kilometer(df, coordinate) < kilometer)].mean()[3:].tolist()
        column_value_arr.append(column_value)
    
    df_add = pd.DataFrame(column_value_arr)
    df_add.columns = column_names
    return pd.concat([bus_stop, df_add], axis=1)

In [ ]:
pd.options.display.max_columns = 100  # 주피터에서 보기편하게 column 늘리기

In [ ]:
kilometer = 0.1

In [ ]:
bus_stop_time = preprocess(flow_time_grouped, bus_stop, kilometer)  # 버스정류장 반경 100m의 유동인구값 평균(시간대별)

100%|██████████| 1179/1179 [00:31<00:00, 37.96it/s]


In [ ]:
bus_stop_age = preprocess(flow_age_grouped, bus_stop, kilometer)  # 버스정류장 반경 100m의 유동인구값 평균(연령대별)

100%|██████████| 1179/1179 [00:30<00:00, 38.25it/s]


## 저장

In [ ]:
bus_stop_time.to_csv('./data/bus_stop_time_0314.csv', index= False)
bus_stop_age.to_csv('./data/bus_stop_age_0314.csv', index= False)